In [67]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(response.text, features="html.parser")

table_data = soup.select('table.wikitable.sortable > tbody > tr')
theads = table_data[0].select('th')
trows = table_data[1:]

columns = []
for thead in theads:
    columns.append(thead.get_text().strip())

data_rows = [[] for col in columns]
for row in trows:
    tds = row.select('td')
    for idx, td in enumerate(tds):
        data_rows[idx].append(td.get_text().strip())

df = pd.DataFrame()
for idx, col in enumerate(columns):
    df[col] = data_rows[idx]

df.to_csv(r'toronto_neighbor.csv', index=False)

In [68]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# !conda install -c conda-forge geocoder --yes

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

address = 'Parkwoods, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.7611243 -79.3240594


In [5]:
def get_pos(address):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    if location is None:
        return [0,0]
    latitude = location.latitude
    longitude = location.longitude
    print(address, latitude, longitude)
    return [latitude, longitude]

In [6]:
df['Neighbourhood_Toronto'] = df['Neighbourhood'] + ', Toronto'
new_cols = df['Neighbourhood_Toronto'].apply(lambda nbh: get_pos(nbh))

Parkwoods, Toronto 43.7611243 -79.3240594
Victoria Village, Toronto 43.732658 -79.3111892
Harbourfront, Toronto 43.6400801 -79.3801495
Regent Park, Toronto 43.6607056 -79.3604569
Lawrence Heights, Toronto 43.7227784 -79.4509332
Lawrence Manor, Toronto 43.7220788 -79.4375067
Islington Avenue, Toronto 43.7192407 -79.5570574
Rouge, Toronto 43.8049304 -79.1658374
Malvern, Toronto 43.8091955 -79.2217008
Don Mills North, Toronto 43.737178 -79.3434514
Woodbine Gardens, Toronto 43.7120785 -79.3025673
Parkview Hill, Toronto 43.7062977 -79.3219073
Ryerson, Toronto 43.65815275 -79.37908010263
Garden District, Toronto 43.6565019 -79.3771275
Glencairn, Toronto 43.7087117 -79.4406853
Cloverdale, Toronto 43.633637 -79.5497447
Islington, Toronto 43.6453376 -79.52487
Martin Grove, Toronto 43.653963 -79.387207
Princess Gardens, Toronto 43.6404664 -79.3912237
West Deane Park, Toronto 43.6631995 -79.5685684
Highland Creek, Toronto 43.7901172 -79.1733344
Rouge Hill, Toronto 43.7802711 -79.1304992
Port Unio

In [69]:
df['Latitude'] = new_cols.apply(lambda x: x[0])
df['Longitude'] = new_cols.apply(lambda x: x[1])
df = df[df['Latitude'] != 0.0]
df = df[df['Longitude'] != 0.0]
df.reset_index(drop=True, inplace=True)
df.shape

(200, 5)

In [70]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.761124,-79.324059
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
3,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
4,M6A,North York,Lawrence Heights,43.722778,-79.450933


In [75]:
ny_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
ny_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
2,M5B,Downtown Toronto,Ryerson,43.658153,-79.379080
3,M5B,Downtown Toronto,Garden District,43.656502,-79.377128
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704


In [76]:
ny_data.shape

(72, 5)

In [78]:
# create map of Manhattan using latitude and longitude values
map_ny = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ny_data['Latitude'], ny_data['Longitude'], ny_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

In [79]:
CLIENT_ID = '2OGJEGMXUMQB4OESKUJ22DTXAZAYAGPYT1OR310CXAULRYDE' # your Foursquare ID
CLIENT_SECRET = 'ZD3ZKJOB2P2DRCSEONN31KSMT4OBCFMTIPQNOF3T3FJIQJBA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2OGJEGMXUMQB4OESKUJ22DTXAZAYAGPYT1OR310CXAULRYDE
CLIENT_SECRET:ZD3ZKJOB2P2DRCSEONN31KSMT4OBCFMTIPQNOF3T3FJIQJBA


In [80]:
ny_data.loc[0, 'Neighbourhood']

'Harbourfront'

In [81]:
neighborhood_latitude = ny_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = ny_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = ny_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6400801, -79.3801495.


In [82]:
search_query = 'York University Heights'
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=2OGJEGMXUMQB4OESKUJ22DTXAZAYAGPYT1OR310CXAULRYDE&client_secret=ZD3ZKJOB2P2DRCSEONN31KSMT4OBCFMTIPQNOF3T3FJIQJBA&v=20180604&ll=43.6655242,-79.3838011&radius=500&limit=100'

In [83]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d173d169b514f0023fdeaf5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Village',
  'headerFullLocation': 'The Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 90,
  'suggestedBounds': {'ne': {'lat': 43.6700242045, 'lng': -79.37759193534978},
   'sw': {'lat': 43.6610241955, 'lng': -79.39001026465021}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51a11515498ef97667980b8c',
       'name': 'DanceLifeX Centre',
       'location': {'address': '619 Yonge Street',
        'crossStreet': 'btwn Isabella St & Gloucester St',
        'lat': 43.66695583448771,
        'lng': -79.3852972984314,
        'labeledL

In [89]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [90]:
ny_venues = getNearbyVenues(names=ny_data['Neighbourhood'],
                            latitudes=ny_data['Latitude'],
                            longitudes=ny_data['Longitude'])

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


In [91]:
print(ny_venues.shape)
ny_venues.head()

(3722, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.64008,-79.38015,Harbour Square Park,43.639253,-79.378395,Park
1,Harbourfront,43.64008,-79.38015,Lake Ontario,43.638945,-79.379665,Lake
2,Harbourfront,43.64008,-79.38015,Harbourfront,43.639526,-79.380688,Neighborhood
3,Harbourfront,43.64008,-79.38015,BeaverTails,43.639899,-79.380197,Bakery
4,Harbourfront,43.64008,-79.38015,Miku,43.641374,-79.377531,Japanese Restaurant


## Clustering

In [106]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighbourhood'] = ny_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
ny_grouped = ny_onehot.groupby('Neighbourhood').mean().reset_index()
ny_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Adelaide,0.00,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.010000,0.000000,0.000000
1,Bathurst Quay,0.00,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Brockton,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.111111,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,CN Tower,0.00,0.000000,0.000000,0.000000,0.000000,0.023529,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.011765,0.000000,0.000000,0.011765,0.000000
5,Cabbagetown,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Chinatown,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.050000,0.010000,0.000000,0.030000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000
8,Christie,0.00,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.016949,0.016949,0.00,0.016949,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.00,0.011111,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011111,0.000000,0.011111,0.00,0.000000,0.011111,0.000000,0.011111,0.000000


In [108]:
num_top_venues = 5

for hood in ny_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
         venue  freq
0  Coffee Shop  0.07
1        Hotel  0.05
2         Café  0.05
3   Restaurant  0.04
4    Gastropub  0.04


----Bathurst Quay----
         venue  freq
0  Coffee Shop  0.17
1         Café  0.12
2         Park  0.08
3          Gym  0.04
4       Garden  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1                Café  0.06
2          Restaurant  0.05
3               Hotel  0.04
4  Italian Restaurant  0.04


----Brockton----
                   venue  freq
0                    Bar  0.17
1                   Park  0.11
2  Vietnamese Restaurant  0.11
3               Dive Bar  0.06
4      French Restaurant  0.06


----CN Tower----
                venue  freq
0         Coffee Shop  0.09
1               Hotel  0.09
2         Pizza Place  0.06
3  Italian Restaurant  0.05
4                 Gym  0.04


----Cabbagetown----
         venue  freq
0   Restaurant  0.10
1  Coffee Shop  0.08
2         Café  0.06
3          Pub  0.0

In [97]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = ny_grouped['Neighbourhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Hotel,Café,Gastropub,Restaurant,Breakfast Spot,Italian Restaurant,Japanese Restaurant,Salad Place,Seafood Restaurant
1,Bathurst Quay,Coffee Shop,Café,Park,Bank,Tunnel,Sushi Restaurant,Sculpture Garden,Diner,Caribbean Restaurant,Ramen Restaurant
2,Berczy Park,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Hotel,Gym,Cocktail Bar,Seafood Restaurant,Beer Bar
3,Brockton,Bar,Vietnamese Restaurant,Park,Dive Bar,Art Gallery,French Restaurant,Gastropub,Korean Restaurant,Bakery,Jazz Club
4,CN Tower,Hotel,Coffee Shop,Pizza Place,Italian Restaurant,Gym,Scenic Lookout,Bar,Baseball Stadium,Aquarium,History Museum


In [114]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [115]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = ny_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

ny_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,0,Coffee Shop,Café,Hotel,Restaurant,Pizza Place,Italian Restaurant,Sports Bar,Steakhouse,Park,Brewery
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,0,Coffee Shop,Thai Restaurant,Pool,Grocery Store,Sushi Restaurant,Fast Food Restaurant,Beer Store,Restaurant,Auto Dealership,Food Truck
2,M5B,Downtown Toronto,Ryerson,43.658153,-79.379080,0,Coffee Shop,Café,Clothing Store,Middle Eastern Restaurant,Ramen Restaurant,Diner,Hotel,Pizza Place,Restaurant,Japanese Restaurant
3,M5B,Downtown Toronto,Garden District,43.656502,-79.377128,0,Coffee Shop,Clothing Store,Cosmetics Shop,Hotel,Café,Middle Eastern Restaurant,Fast Food Restaurant,Restaurant,Pizza Place,Ramen Restaurant
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704,0,Coffee Shop,Pizza Place,Café,Grocery Store,Hostel,Beer Store,Bar,Bus Stop,Library,Market


In [116]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighbourhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [117]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Pizza Place,Italian Restaurant,Sports Bar,Steakhouse,Park,Brewery
1,Downtown Toronto,0,Coffee Shop,Thai Restaurant,Pool,Grocery Store,Sushi Restaurant,Fast Food Restaurant,Beer Store,Restaurant,Auto Dealership,Food Truck
2,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Middle Eastern Restaurant,Ramen Restaurant,Diner,Hotel,Pizza Place,Restaurant,Japanese Restaurant
3,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Hotel,Café,Middle Eastern Restaurant,Fast Food Restaurant,Restaurant,Pizza Place,Ramen Restaurant
4,Downtown Toronto,0,Coffee Shop,Pizza Place,Café,Grocery Store,Hostel,Beer Store,Bar,Bus Stop,Library,Market
5,East Toronto,0,Park,Beach,Breakfast Spot,Pizza Place,Thai Restaurant,Coffee Shop,Japanese Restaurant,Pub,Bar,Tea Room
6,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Hotel,Gym,Cocktail Bar,Seafood Restaurant,Beer Bar
8,Downtown Toronto,0,Korean Restaurant,Coffee Shop,Indian Restaurant,Sandwich Place,Ice Cream Shop,Dessert Shop,Cocktail Bar,Japanese Restaurant,Grocery Store,Café
9,Downtown Toronto,0,Coffee Shop,Hotel,Café,Gastropub,Restaurant,Breakfast Spot,Italian Restaurant,Japanese Restaurant,Salad Place,Seafood Restaurant
10,Downtown Toronto,0,Coffee Shop,Restaurant,Italian Restaurant,Gastropub,Hotel,Café,American Restaurant,Japanese Restaurant,Gym,Farmers Market


In [119]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,1,Campground,Castle,Zoo,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit


In [120]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Central Toronto,2,Bank,Playground,Mediterranean Restaurant,Park,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
35,Central Toronto,2,Bank,Playground,Mediterranean Restaurant,Park,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
55,Central Toronto,2,Bank,Playground,Mediterranean Restaurant,Park,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store


In [121]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Central Toronto,3,Convenience Store,Playground,Tennis Court,Gym,Zoo,Event Space,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store


In [122]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Downtown Toronto,4,Park,Playground,Building,Japanese Restaurant,Zoo,Event Space,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store


## Thanks!